The first notebook to test the idea. 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
#%config InlineBackend.figure_format = 'pdf'

import kgof
import kgof.data as data
import kgof.density as density
import kgof.goftest as gof
import kgof.kernel as ker
import kgof.util as util
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import tensorflow as tf

In [ ]:
# font options
font = {
    #'family' : 'normal',
    #'weight' : 'bold',
    'size'   : 18
}

plt.rc('font', **font)
plt.rc('lines', linewidth=2)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

## Isotropic normal distribution

In [ ]:
d = 1
mean = np.zeros(d)
variance = 5
isonorm = density.IsotropicNormal(mean, variance)

In [ ]:
X = np.linspace(-8, 8, 200)[:, np.newaxis]
#X = np.random.randn(100, d)
grads = isonorm.grad_log(X)
dens = isonorm.log_den(X)

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(X, dens, 'b-', label='log den')
plt.plot(X, grads, 'r-', label='gradient')
plt.grid()
plt.legend(loc='best')

## Test gradient of a kernel

In [ ]:
k = ker.KGauss(sigma2=1)
n = 5
d = 2
with util.NumpySeedContext(seed=2):
    X = np.random.randn(n, d)
    Y = np.random.randn(3, d)

In [ ]:
G = k.grad_x(X, Y)
G

## Test FSSD Test

In [ ]:
# true p
seed = 4
d = 20
mean = np.zeros(d)
variance = 1
isonorm = density.IsotropicNormal(mean, variance)

In [ ]:
# sample
n = 300
draw_variance = variance
# only one dimension of the mean is shifted
#draw_mean = mean + np.hstack((1, np.zeros(d-1)))
draw_mean = mean 
X = util.randn(n, d, seed=seed)*np.sqrt(draw_variance) + draw_mean
dat = data.Data(X)

In [ ]:
# Test
J = 1
alpha = 0.01

sig2 = util.meddistance(X, subsample=1000)**2
k = ker.KGauss(sig2)

# random test locations
V = util.randn(J, d, seed=seed+1) + draw_mean
fssd = gof.FSSD(isonorm, k, V, alpha=alpha, n_simulate=2000, seed=2)

In [ ]:
test_result = fssd.perform_test(dat, return_simulated_stats=True)
test_result

In [ ]:
sim_stats = test_result['sim_stats']
plt.hist(sim_stats, bins=20);